In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import re
import os

In [2]:
def parse_html(file, index):
    answer = {}
    answer['id'] = index
    url_split = file.find('\n')
    url = file[:url_split]
    answer['url'] = url
    
    try:
        text = file[url_split + 1:]
        soup = BeautifulSoup(text, "html5lib")
        if not soup:
            print("Not foud soup! ", index)
            return answer
        name = soup.find('title')
        if name:
            answer['title'] = name.string
        else:
            print("empty title:", index)
        body = soup.find('body')
        if not body:
            print("not found body:", index)
            return answer
        text_prefix = []
        cnt = 0
        try:
            for paragraph in body.find_all(['p', 'h1', 'h2', 'h3']):
                cnt += 1
                if paragraph and paragraph.string:
                    if len(paragraph.string) > 1:
                        text_prefix.append(paragraph.string)
                else:
                    if not paragraph.descendants:
                        print("not found descedants: ", index)
                    try:
                        for child in paragraph.descendants:
                            if child and child.string and len(child.string) > 1:
                                text_prefix.append(child.string)
                    except Exception as e:
                        print(e, file=sys.stderr)
                        print("bad children in file: ", index)
        except Exception as e:
            print(e, file=sys.stderr)
            print("bad find_all in file: ", index)
        rus_text = ' '.join(text_prefix)
        rus_text = re.sub(r"\s+", " ", rus_text)
        answer['text'] = rus_text
        return answer
    except Exception as e:
        print(e, file=sys.stderr)
        print("Unknown error: ", index)

In [3]:
import pandas as pd
import json
import gzip
import codecs
import sys

In [4]:
dirname = 'content'
dir_result = 'data'
meta = []
for index in tqdm(range(8041, 28027)):
    file = str(index) + ".dat"
    with open(os.path.join(dirname, file), mode='r', encoding='utf-8') as fin:
        record = (parse_html(fin.read(), index))
    with gzip.open('data/part_{:05d}.jsonl.gz'.format(index), mode='wb') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)
        try:
            record_str = json.dumps(record, ensure_ascii=False)
            print(record_str, file=f_json)

        except Exception as e:
            print(e, file=sys.stderr)

#     pbar.update(1)    
    
# page_info = pd.DataFrame(meta)
# page_info.to_csv("all_page_info.csv", index=False)

empty title: 9842
not found body: 9842
empty title: 10180
not found body: 10180


'NoneType' object has no attribute 'next_element'


bad children in file:  10956
empty title: 11525
not found body: 11525
empty title: 11724
not found body: 11724
not found body: 11961
not found body: 13200


'NoneType' object has no attribute 'next_element'


bad find_all in file:  13325
empty title: 13949
not found body: 13949


...

bad find_all in file:  19016
not found body: 19788


'NoneType' object has no attribute 'next_element'


bad children in file:  21048
not found body: 21268
empty title: 21931
not found body: 21931
empty title: 22264
not found body: 22264
empty title: 23011
not found body: 23011
empty title: 23070
not found body: 23070


'NoneType' object has no attribute 'next_element'


bad children in file:  25710
empty title: 26823
not found body: 26823



In [7]:
import pandas as pd

page_info = pd.DataFrame(meta)
page_info.to_csv("page_info.csv", index=False)

In [36]:
page_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
text     50 non-null object
title    50 non-null object
url      50 non-null object
dtypes: object(3)
memory usage: 1.2+ KB
